In [1]:
import requests
import time
import pandas as pd
from datetime import datetime
from config import *

In [2]:
# Pull all in all email campaign ids.

url = "https://a.klaviyo.com/api/campaigns/?filter=equals(messages.channel,'email')"
headers = {
    "accept": "application/json",
    "revision": "2023-10-15",
    "Authorization": "Klaviyo-API-Key " + ct_private_key
}

campaigns = []

while url:
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        data = response.json()
        campaigns += [campaign for campaign in data['data'] if campaign['type'] == 'campaign']
        url = data['links'].get('next')  # Get the next page URL
    else:
        print('Failed to retrieve campaigns: ', response.status_code)
        break

# Output is heavily nested, and contains a lot of useless meta data.
# Just going to cherry pick the values we need into a flattened table.
df_campaigns = pd.DataFrame([{
    'campaign_id': campaign['id'],
    'campaign_name': campaign['attributes']['name'],
    'status': campaign['attributes']['status'],
    'audiences': campaign['attributes']['audiences']['included'],
    'send_strategy': campaign['attributes']['send_strategy']['method'],
    'scheduled': campaign['attributes']['scheduled_at'], # Used for metric capture window.
    'sent': campaign['attributes']['send_time'], # Effectively useless depending on smart send times.
} for campaign in campaigns])

# Adding an end date to mark when Klaviyo attribution window ends.
# Will be referenced later when calling for aggregate metrics.
df_campaigns['end_date'] = pd.to_datetime(df_campaigns['sent'])
df_campaigns['end_date'] = df_campaigns['end_date'].apply(lambda x: (x + pd.Timedelta(days=14)).isoformat() if pd.notna(x) else None)

# Limit to only sent emails. No need to pull in Drafts and Upcoming emails.
df_campaigns = df_campaigns[df_campaigns['status'] != 'Draft']
df_campaigns.reset_index(drop=True, inplace=True)

df_campaigns = df_campaigns.head(500)

In [3]:
# Pull all segments.
url = "https://a.klaviyo.com/api/segments/"
headers = {
    "accept": "application/json",
    "revision": "2023-10-15",
    "Authorization": "Klaviyo-API-Key " + ct_private_key
}

# API only allows 10 at a time, need to paginate.

segments = []

while url:
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        data = response.json()
        segments += [segment for segment in data['data'] if segment['type'] == 'segment']
        url = data['links'].get('next')  # Get the next page URL
    else:
        print('Failed to retrieve segments: ', response.status_code)
        break

# Converting to DataFrame
df_segments = pd.DataFrame([{
    'audience_id': segment['id'],
    'audience_name': segment['attributes']['name'],
} for segment in segments])

In [4]:
# Pull all lists.
url = "https://a.klaviyo.com/api/lists/"
headers = {
    "accept": "application/json",
    "revision": "2023-10-15",
    "Authorization": "Klaviyo-API-Key " + ct_private_key
}

# API only allows 10 at a time, need to paginate.

lists = []

while url:
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        data = response.json()
        lists += [list for list in data['data'] if list['type'] == 'list']
        url = data['links'].get('next')  # Get the next page URL
    else:
        print('Failed to retrieve lists: ', response.status_code)
        break

# Converting to DataFrame
df_lists = pd.DataFrame([{
    'audience_id': list['id'],
    'audience_name': list['attributes']['name'],
} for list in lists])

In [5]:
# Combine lists and segments into one data frame.
df_audience = pd.concat([df_segments, df_lists], ignore_index=True)

In [6]:
# Cleaning the 'audiences' column in df_campaigns
df_campaigns['audiences'] = df_campaigns['audiences'].astype(str).str.replace(r'[\[\]\'"]', '', regex=True)

# Splitting the audience IDs into lists, handling None, empty or invalid formats in df_campaigns
df_campaigns['audiences'] = df_campaigns['audiences'].apply(lambda x: x.split(',') if x and x.strip() else [])

# Creating a dictionary from df_audience for mapping
audience_id_name_map = dict(zip(df_audience['audience_id'], df_audience['audience_name']))

# Function to replace IDs with names and concatenate in df_campaigns
def replace_ids_with_names(ids):
    if not ids:
        return None
    names = [audience_id_name_map.get(id.strip()) for id in ids if id.strip() in audience_id_name_map]
    return ', '.join(filter(None, names))  # filter(None, names) removes any None values

# Applying the function to the 'audiences' column in df_campaigns
df_campaigns['audiences'] = df_campaigns['audiences'].apply(replace_ids_with_names)

In [7]:
# Pull all tags
url = "https://a.klaviyo.com/api/tags/"
headers = {
    "accept": "application/json",
    "revision": "2023-10-15",
    "Authorization": "Klaviyo-API-Key " + ct_private_key
}

response = requests.get(url, headers=headers)

# Request test
if response.status_code == 200:
    response = response.json()
else:
    print('Failed to retrieve campaigns: ', response.status_code) 
    
# Extracting metric data
tags = [tag for tag in response['data'] if tag['type'] == 'tag']

# Converting to DataFrame
df_tags = pd.DataFrame([{
    'tag_id': tag['id'],
    'tag_name': tag['attributes']['name'],
    'tag_group_id': tag['relationships']['tag-group']['data']['id']
} for tag in tags])

In [8]:
# Pull all tag groups

url = "https://a.klaviyo.com/api/tag-groups/"
headers = {
    "accept": "application/json",
    "revision": "2023-10-15",
    "Authorization": "Klaviyo-API-Key " + ct_private_key
}

response = requests.get(url, headers=headers)

# Request test
if response.status_code == 200:
    response = response.json()
else:
    print('Failed to retrieve tag groups: ', response.status_code) 
    
# Extracting metric data
tag_groups = [tag_group for tag_group in response['data'] if tag_group['type'] == 'tag-group']

# Converting to DataFrame
df_tag_groups = pd.DataFrame([{
    'tag_group_id': tag_group['id'],
    'tag_group_name': tag_group['attributes']['name']
} for tag_group in tag_groups])

# Pulling in only tag groups we want.
df_tag_groups = df_tag_groups[df_tag_groups['tag_group_name'].isin(['Content', 'Format', 'Program/Audience', 'Sales'])]

In [9]:
# Merge tags with tag groupings.
df_tag_table = df_tags.merge(df_tag_groups, on='tag_group_id', how='left')
# Drop null/old tags that don't have a group assigned.
df_tag_table.dropna(subset=['tag_group_name'], inplace=True)

In [10]:
# Function to call API with entire row and process response
def call_api_for_tags(campaign_row, ct_private_key):
    # Extracting values from the row
    campaign_id = campaign_row['campaign_id']
    campaign_name = campaign_row['campaign_name']

    # API call setup
    url = "https://a.klaviyo.com/api/campaigns/" + campaign_id + "/relationships/tags/"
    headers = {
        "accept": "application/json",
        "revision": "2023-10-15",
        "Authorization": "Klaviyo-API-Key " + ct_private_key
    }

    # Making the API call
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        response = response.json()
        campaign_tags = [tag for tag in response['data'] if tag['type'] == 'tag']
        return pd.DataFrame([{
            'campaign_id': campaign_id,
            'campaign_name': campaign_name,
            'tag_id': tag['id']
        } for tag in campaign_tags])
    else:
        print('Failed to retrieve tags: ', response.status_code)
        return pd.DataFrame()

# DataFrame to store results
df_campaign_tags = pd.DataFrame()

# Iterate over each row in df_campaigns
for index, row in df_campaigns.iterrows():
    df_temp = call_api_for_tags(row, ct_private_key)
    df_campaign_tags = pd.concat([df_campaign_tags, df_temp], ignore_index=True)
    
    # Rate limiting: Burst 3/s, Steady 60/m; appears to be a lie.
    time.sleep(1)

# Merge tags with tag groupings.
df_tag_table = df_campaign_tags.merge(df_tag_table, on='tag_id', how='left')

# Grouping by 'campaign_id' and 'tag_group_name', then joining the 'tag_name's
grouped = df_tag_table.groupby(['campaign_id', 'tag_group_name'])['tag_name'].apply(','.join).reset_index()

# Pivoting the table to get the desired format
pivot = grouped.pivot(index='campaign_id', columns='tag_group_name', values='tag_name').reset_index()

# Merging with df_campaigns
df_campaigns = df_campaigns.merge(pivot, on='campaign_id', how='left')

In [11]:
# Pull in unique metric IDs.

url = "https://a.klaviyo.com/api/metrics/"
headers = {
    "accept": "application/json",
    "revision": "2023-10-15",
    "Authorization": "Klaviyo-API-Key " + ct_private_key
}

response = requests.get(url, headers=headers)
    
# Request test
if response.status_code == 200:
    response = response.json()
else:
    print('Failed to retrieve metrics: ', response.status_code) 
    
# Output is heavily nested, and contains a lot of useless meta data.
# Just going to cherry pick the values we need into a flattened table.
metrics = [item for item in response['data'] if item['type'] == 'metric']

# Converting to DataFrame
df_metrics = pd.DataFrame([{
    'metric_id': metric['id'],
    'metric_name': metric['attributes']['name']
} for metric in metrics])

# ID's below are: received email, opened email, clicked email, unsubscribed.
# Each brand has different/unique metric_id's, need to use the default names to filter what we want.
df_metrics = df_metrics[df_metrics['metric_name'].isin(['Received Email', 'Opened Email', 'Clicked Email', 'Unsubscribed'])]
df_metrics.reset_index(drop=True, inplace=True)

In [12]:
# Klaviyo requires that you call every metric for every campaign.
# Going to have to call every row in df_campaign, and cycle through every df_metric id.
# We could just hard code these in a dict, but I imagine the metric requests will grow with time.
# It will return a daily count of values, but we're going to drop the dates and group aggregate values.
# Should be easier to unpivot in a main table later.

def call_api_for_metrics(campaign_row, metric_row, ct_private_key):
    
    campaign_id = campaign_row['campaign_id']
    campaign_name = campaign_row['campaign_name']
    scheduled = campaign_row['scheduled']
    end_date = campaign_row['end_date']

    metric_id = metric_row['metric_id']
    metric_name = metric_row['metric_name']

    # API URL
    url = "https://a.klaviyo.com/api/metric-aggregates/"

    # Preparing the payload
    payload = { "data": {
            "type": "metric-aggregate",
            "attributes": {
                "metric_id": metric_id,
                "measurements": ["unique"], # Can use: sum_value, count, unique
                "interval": "month", # Can use: hour, day, week, month
                "by": ["$message"],
                "filter": [
                    "greater-or-equal(datetime," + scheduled + ")",  
                    "less-than(datetime," + end_date + ")",  
                    "equals($message,\"" + campaign_id+ "\")"],
                "timezone": "US/Pacific"
            }
        } }

    # Headers for the API request
    headers = {
        "accept": "application/json",
        "revision": "2023-10-15",
        "content-type": "application/json",
        "Authorization": "Klaviyo-API-Key " + ct_private_key
    }

    # Making the API request
    response = requests.post(url, json=payload, headers=headers)

    # Check if the request was successful
    if response.status_code == 200:
        response_data = response.json()
    else:
        print('Failed to retrieve data: ', response.status_code)
        return pd.DataFrame()

    # Extracting metric data
    metric_aggregate = response_data['data']

    # Preparing data for DataFrame
    data_for_df = []
    dates = metric_aggregate['attributes']['dates']
    measurements = metric_aggregate['attributes']['data'][0]['measurements']['unique']

    for date, measurement in zip(dates, measurements):
        data_for_df.append({
            'campaign_id': campaign_id,
            'campaign_name': campaign_name,
            'metric_id': metric_id,
            'metric_name': metric_name,
            'date': date,
            'aggregate': measurement
        })

    return pd.DataFrame(data_for_df)

def main(df_campaigns, df_metrics, ct_private_key):
    final_output = pd.DataFrame()

    for campaign_index, campaign_row in df_campaigns.iterrows():
        for metric_index, metric_row in df_metrics.iterrows():
            result = call_api_for_metrics(campaign_row, metric_row, ct_private_key)
            final_output = pd.concat([final_output, result])

            # Rate limiting: Burst 10/s, Steady 150/m; Actual seems to be 60-75/m.
            time.sleep(0.8)

    return final_output

# Call main()
metric_output = main(df_campaigns, df_metrics, ct_private_key)

# Remove date columns, sum aggregate values to get a readable table.
metric_output = metric_output.groupby(['campaign_id', 'campaign_name', 'metric_id', 'metric_name'])['aggregate'].sum().reset_index()

# Pivot the output, merge into df_campaigns.
pivot = metric_output.pivot(index='campaign_id', columns='metric_name', values='aggregate')
df_campaigns = df_campaigns.merge(pivot, on='campaign_id', how='left')

In [13]:
# test output before push
#df_campaigns.to_csv('Klaviyo_API_CT.csv', index=False)

,campaign_id,campaign_name,status,audiences,send_strategy,scheduled,sent,end_date,Content,Format,Program/Audience,Sales,Clicked Email,Opened Email,Received Email,Unsubscribed
0,01HHZW7WJCE75S5ZKV8C609ZCX,"CBE231224s - Concrete Jungle | 2.5"" Strips",Scheduled,"Expanded Most Engaged, Stash and Save",static,2023-12-19T21:24:02.373395+00:00,2023-12-24T11:00:00+00:00,2024-01-07T11:00:00+00:00,Product Highlight,Long Form,Stash and Save,Item Sale,0.0,0.0,0.0,0.0
1,01HHZQV1D00YJANCJ3JY1KDFX8,"CBE231223s - Sew Great 9"" Classic Fabric Scissors",Scheduled,"Expanded Most Engaged, Stash and Save",static,2023-12-19T19:22:01.642531+00:00,2023-12-23T11:00:00+00:00,2024-01-06T11:00:00+00:00,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0
2,01HHJZCFX3JJ9RWNN8TE7TTDYH,"CBE231219s - Florence's Fancy | 10"" Squares",Sent,"Expanded Most Engaged, Stash and Save",static,2023-12-15T20:16:26.699166+00:00,2023-12-19T11:00:00+00:00,2024-01-02T11:00:00+00:00,Product Highlight,Long Form,Stash and Save,Item Sale,1317.0,70437.0,99625.0,21.0
3,01HHN3XG8EHZYNMWG26PZGKSZS,CBE231222 - Gift Cards,Scheduled,CT 12 month engaged users 10.26.21,static,2023-12-18T20:07:08.486479+00:00,2023-12-22T15:00:00+00:00,2024-01-05T15:00:00+00:00,Product Highlight,Postcard,Standard,No Sale,0.0,0.0,0.0,0.0
4,01HHHWNVA7BAGAMKD8SF35WRRF,CBE231218 - End of Bolt Sale LC,Sent,CT 12 month engaged users 10.26.21,static,2023-12-15T19:14:15.855588+00:00,2023-12-18T19:00:00+00:00,2024-01-01T19:00:00+00:00,Last Chance,Long Form,Standard,Group Sale,4379.0,87623.0,198721.0,55.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,01GRM88KPMAYKKYZ6CRFAA18KP,CBE230218s - Bow Wow Blue,Sent,"Most Engaged 7.19, Stash and Save",static,2023-02-09T23:00:18.227324+00:00,2023-02-18T16:00:00+00:00,2023-03-04T16:00:00+00:00,NaN,NaN,NaN,NaN,1281.0,54134.0,57800.0,20.0
496,01GSB8138XP2682B7V9N7W38FR,CBE230218 - Calico,Sent,CT 12 month engaged users 10.26.21,static,2023-02-17T19:47:43.829771+00:00,2023-02-18T11:00:00+00:00,2023-03-04T11:00:00+00:00,NaN,NaN,NaN,NaN,10674.0,111318.0,205104.0,83.0
497,01GRKRAQDE7WJ8VNKRTHPR7G60,CBE230217s - Hello Daisy,Sent,"Most Engaged 7.19, Stash and Save",static,2023-02-07T23:08:30.172141+00:00,2023-02-17T16:00:00+00:00,2023-03-03T16:00:00+00:00,NaN,NaN,NaN,NaN,1734.0,53041.0,57806.0,19.0
498,01GR7069H3GZRJS9CNQWDXQF2V,CBE230216s - Fairy Frost Frosty 2.5 Strips,Sent,"Most Engaged 7.19, Stash and Save",static,2023-02-07T20:22:25.458315+00:00,2023-02-16T16:00:00+00:00,2023-03-02T16:00:00+00:00,NaN,NaN,NaN,NaN,1357.0,53395.0,57822.0,13.0


In [14]:
from config import *

from sqlalchemy.engine import URL
connection_string = "DRIVER={ODBC Driver 17 for SQL Server};SERVER="+server+";DATABASE="+database+";UID="+user+";PWD="+pwd+";Trusted_Connection=YES"
connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})

from sqlalchemy import create_engine
engine = create_engine(connection_url)

from sqlalchemy.types import NVARCHAR
cursor = engine.connect()

In [15]:
default_length = 255  # You can adjust this as needed
df_campaigns.to_sql(name='campaigns', con=engine, schema='klaviyo', if_exists='replace', index=False, 
                    dtype={column_name: NVARCHAR(default_length) for column_name in df_campaigns.columns})

107